In [1]:
import pandas as pd
from bs4 import BeautifulSoup

C:\Users\meshari\anaconda3\envs\Graduation-Project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\meshari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<img src="files/image.png">

we can see from the picture below the structure of the data, what we are intrested in is the target which are the aspcet terms to compare with the extracted aspcet using the algorithm

In [5]:
# Reading the data inside the xml
with open('EN_REST_SB1_TEST.xml.gold', 'r') as f:
    data = f.read()
    
# Passing the stored data inside
# the beautifulsoup parser
Bs_data = BeautifulSoup(data, "lxml")
 
# Finding all instances of tag `opinion`
opinions = Bs_data.find_all('opinions')
# extracting the last opinion for a sentence 
# and getting the targets (aspects) values for the that opinion
aspects = [x.find_all("opinion")[-1]["target"] if len(x) > 0 else None for x in opinions]
# getting polarity (y values) 
y_test = [x.find_all("opinion")[-1]["polarity"] if len(x) > 0 else None for x in opinions if len(x) > 0 ]

In [6]:
aspects = [x.find_all("opinion") if len(x) > 0 else None for x in opinions]
print(len(aspects))
aspects_ = []
for aspect in aspects:
    if aspect:
        aspect_ = []
        for asp in aspect:
            aspect_.append(asp['target'])
        aspects_.append(aspect_)
    else:
        aspects_.append([])
aspects_

676


[['NULL'],
 ['sushi'],
 ['portions'],
 ['Green Tea creme brulee'],
 ['NULL'],
 ['NULL'],
 ['place'],
 ['sushi', 'service'],
 ['staff'],
 ['restaurant'],
 ['NULL'],
 ['NULL'],
 ['owner'],
 ['food'],
 [],
 ['meal'],
 ['NULL'],
 ['staff'],
 ['onion rings'],
 ['NULL'],
 ['NULL'],
 ['food'],
 ['lemon chicken', 'honey walnut prawns', 'honey walnut prawns'],
 [],
 ['ambience', 'place'],
 ['NULL'],
 ['Service'],
 [],
 ['service'],
 ['waiter'],
 ['manager'],
 ['NULL'],
 ['Italian Food'],
 ['Mioposto'],
 ['Italian restaurant'],
 ['wine list', 'food'],
 ['restaurant'],
 ['meal', 'service', 'ambiance'],
 ['NULL'],
 ['wine list', 'food', 'staff'],
 ['place'],
 ['NULL'],
 ['NULL'],
 ['NULL'],
 [],
 [],
 ['food'],
 ['food', 'food'],
 ['Kafta plate'],
 ['NULL'],
 ['meal'],
 ['food', 'service'],
 ['place', 'food'],
 ['food'],
 [],
 [],
 ['NULL'],
 ['NULL'],
 ['atmosphere', 'food', 'food'],
 ['food'],
 ['Rice', 'tuna'],
 ['NULL'],
 ['NULL'],
 ['fish', 'fish'],
 ['service', 'NULL'],
 [],
 ['fish', 'rolls

In [7]:
aspects[:10]

[[<opinion category="FOOD#QUALITY" from="0" polarity="positive" target="NULL" to="0"></opinion>],
 [<opinion category="FOOD#QUALITY" from="19" polarity="positive" target="sushi" to="24"></opinion>],
 [<opinion category="FOOD#STYLE_OPTIONS" from="16" polarity="neutral" target="portions" to="24"></opinion>],
 [<opinion category="FOOD#QUALITY" from="0" polarity="positive" target="Green Tea creme brulee" to="22"></opinion>],
 [<opinion category="FOOD#QUALITY" from="0" polarity="positive" target="NULL" to="0"></opinion>],
 [<opinion category="RESTAURANT#GENERAL" from="0" polarity="positive" target="NULL" to="0"></opinion>],
 [<opinion category="RESTAURANT#GENERAL" from="32" polarity="positive" target="place" to="37"></opinion>],
 [<opinion category="FOOD#QUALITY" from="13" polarity="positive" target="sushi" to="18"></opinion>,
  <opinion category="SERVICE#GENERAL" from="35" polarity="positive" target="service" to="42"></opinion>],
 [<opinion category="SERVICE#GENERAL" from="11" polarity="po

In [4]:
# aspects = Bs_data.find_all('opinion')
# # aspects = [x["target"] for x in aspects]
# print(aspects[1])

In [5]:
# data = pd.read_json('google_balanced.json')
# data = google_clean_reviews(data)
# reviews = list(data['text'])

In [8]:
# gettin all text tags
reviews = Bs_data.find_all('text', text=True)
# extracting only the text from the text tags
reviews = [x.text for x in reviews]
# aspects = [x["target"] for x in aspects]
print(reviews[0])

Yum!


In [9]:
len(reviews)

676

In [11]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [23]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            target.append(token.text)
    pred.append(set(target))
print(pred[:10])

[set(), set(), set(), {'brulee'}, set(), set(), set(), set(), {'staff'}, set()]


### testing the result without the testing null values

In [24]:
overall_correct = 0
pred_list = []
for x in pred:
    pred_list.extend(list(x))
len_pred = len(pred_list)

for i in range(len(aspects_)):
    review_correct = len(pred[i].intersection(aspects_[i]))
    overall_correct += review_correct

print(f'correct aspects: {overall_correct}')    
print(f'extra incorrect aspects: {len_pred-overall_correct}')
print(f'empty extracted aspects: {pred.count(set())}')

correct aspects: 157
extra incorrect aspects: 187
empty extracted aspects: 410


### testing the result with null values

In [15]:
pred[0] = pred[0].replace('', 'NULL')

in over all level it seems there is a lot of the aspcets being empty and we can see that from the exracted sample with 8 from the first 10 aspects being empty, with the test results without converting the empty values to null and thus not counting them we can see that the algorithm got only around 100 from a total of close to 700, and when take empty values into considration the number doubles, since most of the aspects wasn't caught in the first place with this strict algorithm, and that's because of the use of dependnce tag of nsubj, although it limits getting out of bound aspects but it also miss a lot of possible aspcects as a result.

In [20]:
null_values = len(pred[pred.iloc[:,0]=='NULL'])
non_null = len(pred) - null_values

print(f'number of null values = {null_values}')
print(f'number of non null values = {non_null}')
print(f'precentage of correctly extracted aspects = {correct_no_null / non_null * 100 : .2f}%')
print(f'precentage of correctly extracted aspects without accounting for null values = {correct_null / len(pred) * 100 : .2f}%')

number of null values = 420
number of non null values = 256
precentage of correctly extracted aspects =  39.45%
precentage of correctly extracted aspects without accounting for null values =  38.61%


indeed this algorithm is too strict 419 out of the 676 is empty, which amounts to around 75% of the data, so let's see what's the results without using the nsubj dependnce tag.

In [262]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for token in doc:
        if token.pos_ == 'NOUN':
            target.append(token.text)
    pred.append(set(target))
print(pred[:10])

[set(), {'sushi'}, {'portions'}, {'creme', 'must'}, {'restaurant'}, {'Comparison'}, {'place'}, {'service', 'sushi'}, {'staff', 'need'}, {'dozen', 'date', 'restaurant', 'times', 'complaints'}]


In [263]:
overall_correct = 0
pred_list = []
for x in pred:
    pred_list.extend(list(x))
len_pred = len(pred_list)

for i in range(len(aspects_)):
    review_correct = len(pred[i].intersection(aspects_[i]))
    overall_correct += review_correct

print(f'correct aspects: {overall_correct}')    
print(f'extra incorrect aspects: {len_pred-overall_correct}')
print(f'empty extracted aspects: {pred.count(set())}')

correct aspects: 398
extra incorrect aspects: 1295
empty extracted aspects: 85


### testing the result without the counting null values

In [71]:
pred = pd.DataFrame(pred)

In [72]:
r = pred == aspects

In [73]:
r.value_counts()

False    525
True     151
dtype: int64

wee can see that the results improved from 101 to 151 without using the dependnce tag of nsubj, but we need to take into consideration the loose standred of this algorithm since it takes any noun as an aspect.

In [74]:
# saving the correct values to be used later
correct_no_null = r.value_counts()[True].iloc[0]

### testing the result with null values

In [75]:
pred[0] = pred[0].replace('', 'NULL')

In [27]:
# pred.iloc[0] == aspects.iloc[0]

In [76]:
r = pred == aspects

In [77]:
r.value_counts()

False    467
True     209
dtype: int64

In [67]:
# saving the correct values to be used later
correct_null = r.value_counts()[True].iloc[0]

In [69]:
null_values = len(pred[pred.iloc[:,0]=='NULL'])
non_null = len(pred) - null_values

print(f'number of null values = {null_values}')
print(f'number of non null values = {non_null}')
print(f'accuracy of aspect extraction algorithm  = {correct_no_null / non_null * 100 : .2f}%')
print(f'accuracy of aspect extraction algorithm without accounting for null values = {correct_null / len(pred) * 100 : .2f}%')

number of null values = 85
number of non null values = 591
accuracy of aspect extraction algorithm  =  35.36%
accuracy of aspect extraction algorithm without accounting for null values =  30.92%


only 85 values is with empty aspects, but as a drawback out of the extracted aspects only 35.36% of them was correct, compared to using nsubj dependnce tag which give 39.45% with an adavantage of , and an even bigger diffrence when we take into account the null value wa can see that the use of the msubj dependnce tag have a around 8% of an advantage, finally we can see that using nsubj dependnce tag gives better quality.

in the end using nsubj dependnce tag, give more precise results but with more missing aspects, in the other hand without using nsubj dependnce tag, it gives a higher number of non empty aspects with a lower accurate results.

as a final note: the noun taking into consideration is the last found noun, if it's possible to filter the noun in a better way it should give a better results.

In [270]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]
            
        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            target.append(word2.text)
            continue
            
        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            target.append(word3.text)
            continue

        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            target.append(word3.text)
            continue
            
        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            target.append(word3.text)
            continue

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'VBD' or word2.tag_ == 'VBN' or word2.tag_ == 'VBG'):
            target.append(word2.text)
            continue
            
    pred.append(set(target))

In [356]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]
            
        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS'):
            target.append(word1.text)
                
        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            
            
        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word3.text)
            

        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word3.text)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'VBD' or word2.tag_ == 'VBN' or word2.tag_ == 'VBG'):
            word = word1.text+' '+word2.text
            target.append(word)
            
            
    pred.append(set(target))

In [361]:
overall_correct = 0
pred_list = []
for x in pred:
    pred_list.extend(list(x))
len_pred = len(pred_list)

for i in range(len(aspects_)):
    review_correct = len(pred[i].intersection(aspects_[i]))
    overall_correct += review_correct

print(f'correct aspects: {overall_correct}')    
print(f'extra incorrect aspects: {len_pred-overall_correct}')
print(f'empty extracted aspects: {pred.count(set())}')
print(f'unique aspects : {len(set(pred_list))}')

correct aspects: 406
extra incorrect aspects: 2163
empty extracted aspects: 71
unique aspects : 1480


### testing the result with null values

In [104]:
pred[0] = pred[0].replace('', 'NULL')

In [361]:
overall_correct = 0
pred_list = []
for x in pred:
    pred_list.extend(list(x))
len_pred = len(pred_list)

for i in range(len(aspects_)):
    review_correct = len(pred[i].intersection(aspects_[i]))
    overall_correct += review_correct

print(f'correct aspects: {overall_correct}')    
print(f'extra incorrect aspects: {len_pred-overall_correct}')
print(f'empty extracted aspects: {pred.count(set())}')
print(f'unique aspects : {len(set(pred_list))}')

correct aspects: 406
extra incorrect aspects: 2163
empty extracted aspects: 71
unique aspects : 1480


In [358]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]
          
        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            target.append(word2.text)
            continue

        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            target.append(word3.text)
            continue

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ'):
            target.append(word2.text)
            continue

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            target.append(word3.text)
            continue

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'VBN' or word2.tag_ == 'VBD'):
            target.append(word2.text)
            continue

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'JJ'):
            target.append(word3.text)
            continue

        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            target.append(word2.text)
            continue

        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS'):
            target.append(word2.text)
            continue
            
    pred.append(set(target))     

In [359]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]
 
        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS'):
            target.append(word1.text)
            
        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'VBN' or word2.tag_ == 'VBD'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'JJ'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS'):
            word = word1.text+' '+word2.text
            target.append(word)
            
            
    pred.append(set(target))     

In [360]:
overall_correct = 0
pred_list = []
for x in pred:
    pred_list.extend(list(x))
len_pred = len(pred_list)

for i in range(len(aspects_)):
    review_correct = len(pred[i].intersection(aspects_[i]))
    overall_correct += review_correct

print(f'correct aspects: {overall_correct}')    
print(f'extra incorrect aspects: {len_pred-overall_correct}')
print(f'empty extracted aspects: {pred.count(set())}')
print(f'unique aspects : {len(set(pred_list))}')

correct aspects: 406
extra incorrect aspects: 2163
empty extracted aspects: 71


### testing the result with null values

In [114]:
pred[0] = pred[0].replace('', 'NULL')

In [307]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]

        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS'):
            target.append(word1.text)
            continue


        if (word1.tag_ == 'VB'):
            target.append(word1.text)
            continue


        if (word1.tag_ == 'DT' or word1.tag_ == 'JJ' or word1.tag_ == 'NN') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            target.append(word2.text)
            continue


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS' or word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'VBN' or word2.tag_ == 'VBD'):
            target.append(word2.text)
            continue


        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS'):
            target.append(word2.text)
            continue


        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            target.append(word3.text)
            continue


        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS' or word3.tag_ == 'JJ'):
            target.append(word3.text)
            continue


        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'VBN' or word2.tag_ == 'VBD') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            target.append(word3.text)
            continue


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'IN' or word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            target.append(word3.text)
            continue


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'IN') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            target.append(word3.text)
            continue


        if (word1.tag_ == 'NN') and \
           (word2.tag_ == 'TO') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            target.append(word3.text)
            continue
            
    pred.append(set(target)) 

In [362]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]

        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS'):
            target.append(word1.text)
            


        if (word1.tag_ == 'VB'):
            target.append(word1.text)
            


        if (word1.tag_ == 'DT' or word1.tag_ == 'JJ' or word1.tag_ == 'NN') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS' or word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'VBN' or word2.tag_ == 'VBD'):
            word = word1.text+' '+word2.text
            target.append(word)
            


        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS'):
            word = word1.text+' '+word2.text
            target.append(word)
            


        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS' or word3.tag_ == 'JJ'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'VBN' or word2.tag_ == 'VBD') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'IN' or word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'IN') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'NN') and \
           (word2.tag_ == 'TO') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            
            
    pred.append(set(target)) 

In [374]:
overall_correct = 0
pred_list = []
for x in pred:
    pred_list.extend(list(x))
len_pred = len(pred_list)

for i in range(len(aspects_)):
    review_correct = len(pred[i].intersection(aspects_[i]))
    overall_correct += review_correct

print(f'correct aspects: {overall_correct}')    
print(f'extra incorrect aspects: {len_pred-overall_correct}')
print(f'empty extracted aspects: {pred.count(set())}')
print(f'unique aspects : {len(set(pred_list))}')
print(f'Precision: {overall_correct/(overall_correct+(len_pred-overall_correct))}')
aspect_list = []
for x in aspects_:
    aspect_list.extend(list(x))
    
print(f'unique correct aspects : {len(set(pred_list).intersection(aspect_list))}')

correct aspects: 470
extra incorrect aspects: 3486
empty extracted aspects: 49
unique aspects : 2373
Precision: 0.905587668593449
Precision: 0.11880687563195147
unique correct aspects : 203


In [375]:
pred = []
for review in reviews:
    doc = nlp(review)
    target = []
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]
            
        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS'):
            target.append(word1.text)
                
        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            
            
        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word3.text)
            

        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word3.text)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'VBD' or word2.tag_ == 'VBN' or word2.tag_ == 'VBG'):
            word = word1.text+' '+word2.text
            target.append(word)
            
        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'VBN' or word2.tag_ == 'VBD'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'JJ'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            

        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            

        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS'):
            word = word1.text+' '+word2.text
            target.append(word)
            
        if (word1.tag_ == 'VB'):
            target.append(word1.text)
            


        if (word1.tag_ == 'DT' or word1.tag_ == 'JJ' or word1.tag_ == 'NN') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            word = word1.text+' '+word2.text
            target.append(word)
            


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS' or word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'VBN' or word2.tag_ == 'VBD'):
            word = word1.text+' '+word2.text
            target.append(word)
            


        if (word1.tag_ == 'VBN' or word1.tag_ == 'VBD') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS'):
            word = word1.text+' '+word2.text
            target.append(word)
            


        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ' or word2.tag_ == 'RB' or word2.tag_ == 'RBR' or word2.tag_ == 'RBS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS' or word3.tag_ == 'JJ'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'VBN' or word2.tag_ == 'VBD') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'IN' or word2.tag_ == 'NN' or word2.tag_ == 'NNS') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'IN') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)
            


        if (word1.tag_ == 'NN') and \
           (word2.tag_ == 'TO') and \
           (word3.tag_ == 'NN' or word3.tag_ == 'NNS'):
            word = word1.text+' '+word2.text+' '+word3.text
            target.append(word)            
            
    pred.append(set(target))

In [376]:
overall_correct = 0
pred_list = []
for x in pred:
    pred_list.extend(list(x))
len_pred = len(pred_list)

for i in range(len(aspects_)):
    review_correct = len(pred[i].intersection(aspects_[i]))
    overall_correct += review_correct

print(f'correct aspects: {overall_correct}')    
print(f'extra incorrect aspects: {len_pred-overall_correct}')
print(f'empty extracted aspects: {pred.count(set())}')
print(f'unique aspects : {len(set(pred_list))}')
print(f'Precision: {overall_correct/(overall_correct+(len_pred-overall_correct))}')
aspect_list = []
for x in aspects_:
    aspect_list.extend(list(x))
    
print(f'unique correct aspects : {len(set(pred_list).intersection(aspect_list))}')

correct aspects: 471
extra incorrect aspects: 3736
empty extracted aspects: 47
unique aspects : 2594
Precision: 0.11195626337057285
unique correct aspects : 204


### testing the result with null values

In [148]:
pred[0] = pred[0].replace('', 'NULL')

In [ ]:
pred = []
for sentence in reviews:
    doc = nlp(sentence)
    descriptive_term = ''
    target = ''
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            target = token.text
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = prepend + token.text
    pred.append(target)
print(pred[:10])

In [ ]:
pred = []
for sentence in reviews:
    doc = nlp(sentence)
    descriptive_term = ''
    target = ''
    for token in doc:
        if token.pos_ == 'NOUN':
            target = token.text
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = prepend + token.text
    pred.append(target)
print(pred[:10])

# old code

In [ ]:
r = list(r[0])
r = [not x for x in r]

In [ ]:
f_pred = pred[r]
f_asp = aspects[r]
f_rev = [reviews[x]  for x in range(len(reviews)) if r[x] == True]

In [ ]:
f_pred

In [ ]:
f_rev

In [ ]:
f_asp

In [ ]:
aspects

In [ ]:
for review in reviews:
    doc = nlp(review)
    target = ''
    for i in range(len(doc)):
        if i != 0:
            word1 = doc[i-1]
        else:
            word1 = nlp(' ')[0]
        word2 = doc[i]
        if i+1 != len(doc):
            word3 = doc[i+1]
        else:
            word3 = nlp(' ')[0]
    #     word1 = doc[i] 
    #     word2 = doc[i-1]
    #     word3 = doc[i+1]
    #     print(word1.tag_)
        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'NN' or word2.tag_ == 'NNS'):
            print('rule1')
            print(word1, word2)

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            print('rule2')
            print(word1, word2, word3)

        if (word1.tag_ == 'JJ') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            print('rule3')
            print(word1, word2, word3)

        if (word1.tag_ == 'NN' or word1.tag_ == 'NNS') and \
           (word2.tag_ == 'JJ') and \
           (word3.tag_ != 'NN' or word3.tag_ != 'NNS'):
            print('rule4')
            print(word1, word2, word3)

        if (word1.tag_ == 'RB' or word1.tag_ == 'RBR' or word1.tag_ == 'RBS') and \
           (word2.tag_ == 'VB' or word2.tag_ == 'VBD' or word2.tag_ == 'VBN' or word2.tag_ == 'VBG'):
            print('rule5')
            print(word1, word2)
#     if token.pos_ == 'NOUN':
#         target = token.text
# pred.append(target)
# print(pred[:10])

In [18]:
len(pred)

676

In [21]:
aspects_

[[<opinion category="FOOD#QUALITY" from="0" polarity="positive" target="NULL" to="0"></opinion>],
 [<opinion category="FOOD#QUALITY" from="19" polarity="positive" target="sushi" to="24"></opinion>],
 [<opinion category="FOOD#STYLE_OPTIONS" from="16" polarity="neutral" target="portions" to="24"></opinion>],
 [<opinion category="FOOD#QUALITY" from="0" polarity="positive" target="Green Tea creme brulee" to="22"></opinion>],
 [<opinion category="FOOD#QUALITY" from="0" polarity="positive" target="NULL" to="0"></opinion>],
 [<opinion category="RESTAURANT#GENERAL" from="0" polarity="positive" target="NULL" to="0"></opinion>],
 [<opinion category="RESTAURANT#GENERAL" from="32" polarity="positive" target="place" to="37"></opinion>],
 [<opinion category="FOOD#QUALITY" from="13" polarity="positive" target="sushi" to="18"></opinion>,
  <opinion category="SERVICE#GENERAL" from="35" polarity="positive" target="service" to="42"></opinion>],
 [<opinion category="SERVICE#GENERAL" from="11" polarity="po

In [22]:
# pred = pd.DataFrame(pred)
# aspects = pd.DataFrame(aspects_)

In [12]:
r = pred == aspects

In [13]:
r.value_counts()

False    575
True     101
dtype: int64

In [14]:
# saving the correct values to be used later
correct_no_null = r.value_counts()[True].iloc[0]

In [108]:
null_values = len(pred[pred.iloc[:,0]=='NULL'])
non_null = len(pred) - null_values

print(f'number of null values = {null_values}')
print(f'number of non null values = {non_null}')
print(f'accuracy of aspect extraction algorithm  = {correct_no_null / non_null * 100 : .2f}%')
print(f'accuracy of aspect extraction algorithm without accounting for null values = {correct_null / len(pred) * 100 : .2f}%')

number of null values = 229
number of non null values = 447
accuracy of aspect extraction algorithm  =  11.41%
accuracy of aspect extraction algorithm without accounting for null values =  17.90%
